# Regressione lineare e polinomiale con scikit-learn

**Programmazione di Applicazioni Data Intensive**  
Laurea in Ingegneria e Scienze Informatiche  
DISI - Università di Bologna, Cesena

Proff. Gianluca Moro, Roberto Pasolini  
`nome.cognome@unibo.it`

## Setup

Importare i package necessari per verificarne l'installazione.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
%matplotlib inline

## Ripasso: Regressione lineare

Un _modello di regressione_ fornisce una stima $\hat{y}$ del valore $y$ di una variabile _dipendente_ basata sui valori $x_1,\ldots,x_n$ di una o più variabili _indipendenti_.

In un modello di regressione _lineare_, il valore della variabile dipendente è previsto come combinazione lineare di quelli delle variabili indipendenti

- ciascuna variabile indipendente $x_i$ è moltiplicata per un coefficiente $\theta_i$
- viene aggiunto un termine noto (_intercetta_) $\theta_0$

$$ \hat{y} = \theta_0+\theta_1\cdot x_1+\ldots+\theta_n\cdot x_n $$

Nella scorsa lezione, abbiamo visto come implementare la _discesa del gradiente_ per addestrare un modello di regressione lineare su dati esistenti

Vediamo oggi come usare invece una libreria che fornisce questa funzione e altre più avanzate...

## scikit-learn

scikit-learn offre diversi algoritmi e funzioni di supporto per l'addestramento di modelli predittivi, dalla regressione lineare ad altri più avanzati

- tutti i tipi di modelli offrono un'interfaccia comune per l'addestramento e la predizione (inferenza)
- i dati in ingresso sono rappresentati come array NumPy o serie/frame pandas
- sono fornite funzionalità di supporto (suddivisione dati, normalizzazione, ...) compatibili con tutti i tipi di modelli

## Caso di studio 1: predizione consumo elettricità

Carichiamo i dati già visti nel laboratorio precedente: per ogni giorno degli anni dal 2015 al 2017 abbiamo la temperatura media in una città e il picco registrato di consumo di corrente elettrica

- con l'opzione `index_col` specifichiamo che la colonna `date` costituisce l'indice del DataFrame
- con `parse_dates` indichiamo che i suoi valori vanno interpretati come date

In [ ]:
POWER_DATA_URL = "https://github.com/datascienceunibo/dialab2024/raw/main/Regressione_non_Lineare/power.csv"

import os.path
if not os.path.exists("power.csv"):
    from urllib.request import urlretrieve
    urlretrieve(POWER_DATA_URL, "power.csv")

In [ ]:
power = pd.read_csv("power.csv", index_col="date", parse_dates=["date"])

In [ ]:
power.head(8)

### Predizione nei mesi estivi

Nella scorsa esercitazione, abbiamo estratto un modello di predizione sui consumi dei mesi estivi, imputabili all'aria condizionata

Come la scorsa volta, selezioniamo dal frame caricato solamente le righe relative ai mesi di giugno, luglio e agosto

In [ ]:
power_summer = power.iloc[power.index.month.isin([6, 7, 8])]
power_summer

## Addestramento di un modello

Vediamo come addestrare un modello di regressione lineare su questi dati.

Per prima cosa, va creato un oggetto che rappresenta il modello "vuoto", non ancora addestrato.

A seconda del tipo di modello da addestrare, utilizziamo una classe diversa: per un modello di regressione lineare semplice (univariata o multivariata), creiamo un oggetto `LinearRegression`, importandolo dal package dei modelli lineari.

In [ ]:
from sklearn.linear_model import LinearRegression
lrm = LinearRegression()

Usiamo quindi il metodo `fit` per addestrare il modello su un insieme di dati fornito.

Al metodo `fit` vanno passati come argomenti:

- una matrice (o DataFrame) di forma $m\times n$, contenente $m$ osservazioni di $n$ variabili indipendenti $X_1,\ldots,X_n$;
- un vettore (o serie) di $m$ elementi, contenenti i corrispondenti valori noti della variabilie dipendente $Y$.

Nel caso di regressione univariata con una variabile indipendente $X$, va passata una matrice $m\times 1$ per esplicitare che si tratta di $m$ osservazioni di una variabile (piuttosto che di una osservazione di $n$ variabili).

Selezionando la colonna `temp` come una lista (`[["temp"]]` invece di `["temp"]`), ottengo un DataFrame $m\times 1$ invece di una semplice serie.

In [ ]:
X = power_summer[["temp"]]
y = power_summer["demand"]

In [ ]:
X.head()

In [ ]:
y.head()

`X` e `y` possono essere passate quindi come argomenti al metodo `fit`.

Il metodo `fit` restituisce il modello stesso, che in versioni recenti di scikit-learn/Jupyter viene visualizzato in forma di diagramma (questo sarà utile per rappresentare modelli più complessi).

In [ ]:
lrm.fit(X, y)

`lrm` è ora un modello addestrato e può essere utilizzato per effettuare predizioni.

## Inferenza

Una volta che il modello è addestrato, possiamo ottenere predizioni col metodo `predict`:

- va data in input una matrice o frame $m\times n$ di osservazioni delle variabili indipendenti;
- è restituito un vettore di $m$ valori della variabile dipendente previsti dal modello.

Ad esempio, passiamo gli stessi dati usati per l'addestramento per vedere le predizioni del modello.

In [ ]:
preds = lrm.predict(X)

Vediamo alcuni dei valori predetti

In [ ]:
preds[:5]

`predict` restituisce **sempre** un array NumPy. Per ottenere una serie pandas è necessario applicare manualmente le etichette, ad es. copiando quelle delle righe del frame dato in input.

In [ ]:
preds = pd.Series( lrm.predict(X), index=power_summer.index )

In [ ]:
preds.head()

## Parametri del modello

Una volta addestrato il modello possiamo consultarne i parametri tramite due attributi:

- `coef_` restituisce un array dei coefficienti angolari (uno solo nel caso univariato);
- `intercept_` restituisce il valore dell'intercetta.

In [ ]:
lrm.coef_

In [ ]:
lrm.intercept_

Questi sono i valori di $\alpha$ e $\beta$ nella formula del modello lineare univariato $h(x)=\alpha\cdot x+\beta$.

Il metodo `predict` su una matrice `X` in pratica effettua il seguente calcolo:

In [ ]:
preds = X.values @ lrm.coef_ + lrm.intercept_

## Misurare l'errore del modello

Per misurare l'errore che un modello $h(\mathbf{x})$ compie su un set di dati abbiamo usato nello scorso laboratorio l'_errore quadratico medio_ (_mean squared error_, MSE).

$$ \mathrm{MSE} = \frac{1}{m}\sum_{i=1}^n\left(h(\mathbf{x}_i)-y_i\right)^2 $$

Possiamo calcolarlo applicando la formula ai valori reali `y` e a quelli dati dal modello `preds`.

In [ ]:
np.mean(np.square(preds - y))

In alternativa, possiamo usare l'apposita funzione `mean_squared_error` fornita da scikit-learn nel modulo `metrics`.

In [ ]:
from sklearn.metrics import mean_squared_error

Alla funzione vanno passati come argomenti:

- il vettore (o serie) dei valori reali della variabile dipendente,
- il vettore dei corrispondenti valori predetti.

In [ ]:
mean_squared_error(y, preds)

L'errore è leggermente inferiore rispetto a quello ottenuto in precedenza con la discesa gradiente: questo perché scikit-learn calcola i parametri che minimizzano l'errore in modo analitico, per cui la soluzione sarà sempre migliore di (o uguale a) quella ottenuta con la discesa gradiente.

La discesa gradiente torna però utile in problemi più complessi non risolvibili analiticamente, ad es. l'addestramento di reti neurali.

### Errore relativo

Il MSE è utilizzato per confrontare diversi modelli addestrati sugli stessi dati. Si usa in particolare in fase di addestramento, in quanto è facile estrarne la derivata.

Esistono però altre metriche di errore, più facilmente interpretabili.

L'_errore relativo_ misura intuitivamente di quanto il modello si sbaglia in percentuale rispetto al valore reale.

$$ \mathrm{errore} = \frac{1}{m}\sum_{i=1}^m\left\vert\frac{h(\mathbf{x}_i)-y_i}{y_i}\right\vert $$

Come esempio su un singolo caso, se il valore reale da predire è 2 GW, ad una stima di 2,2 GW corrisponde un errore relativo del 10\%.

Definiamo una funzione per il calcolo, compatibile con quella usata sopra per il MSE.

In [ ]:
def relative_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true))

L'errore relativo del modello sopra è:

In [ ]:
relative_error(y, preds)

Ovvero, le predizioni del modello hanno un errore medio del 5,68\% rispetto al valore reale.

### Coefficiente di determinazione

Il _coefficiente di determinazione_ $R^2$ è una misura che esprime intuitivamente quanto il modello catturi la variabilità presente nei dati ($\bar{y}$ = media di $Y$).

$$ R^2 = \frac{\sum_{i=1}^m{\left(y_i-\hat{y}_i\right)^2}}{\sum_{i=1}^m{\left(y_i-\bar{y}\right)^2}} $$

Il coefficiente $R^2$ è compreso tra 1 (il modello descrive perfettamente i dati) e 0 (nessuna relazione tra modello e dati).

Questa metrica si può calcolare con la funzione `r2_score` del modulo `metrics` citato sopra.

In [ ]:
from sklearn.metrics import r2_score
r2_score(y, preds)

Essendo la metrica di default per i modelli di regressione, si può anche usare il metodo `score` del modello passando le osservazioni delle variabili indipendenti e dipendente su cui calcolarla.

Le predizioni non sono passate al metodo score perché sono calcolate usando `X`. `model.score(X, y)` equivale in pratica a `r2_score(y, model.predict(X))`.

In [ ]:
lrm.score(X, y)

Per comodità, creiamo una funzione che calcoli e stampi tutte e tre le metriche su dati e modello forniti.

_(`{x:.5}` indica di stampare il valore `x` con 5 cifre decimali, con `%` lo si stampa in forma percentuale)_

In [ ]:
def print_eval(X, y, model):
    preds = model.predict(X)
    mse = mean_squared_error(y, preds)
    re = relative_error(y, preds)
    r2 = r2_score(y, preds)
    print(f"   Mean squared error: {mse:.5}")
    print(f"       Relative error: {re:.5%}")
    print(f"R-squared coefficient: {r2:.5}")

Testiamo la funzione sul modello sopra.

In [ ]:
print_eval(power_summer[["temp"]], power_summer["demand"], lrm)

## Visualizzazione del modello

Definiamo una funzione che, dato un modello basato su una variabile indipendente, mostri la funzione descritta dal modello sovrapposta ai dati

Si tratta della stessa funzione usata nella scorsa esercitazione, adattata per usare modelli di scikit-learn piuttosto che funzioni generiche.

- Del modello passato viene invocato il metodo `predict`
- La funzione `wrap_vector_in_frame` incapsula un vettore di valori di `x` in un DataFrame simile a quello dei dati originali (non è strettamente necessario, ma evita di visualizzare un warning)

In [ ]:
def plot_model_on_data(X, y, model=None):
    plt.figure(figsize=(10, 7))
    plt.scatter(X, y)
    if model is not None:
        xlim, ylim = plt.xlim(), plt.ylim()
        line_x = np.linspace(xlim[0], xlim[1], 100)
        line_x_df = pd.DataFrame(line_x[:, None], columns=X.columns)
        line_y = model.predict(line_x_df)
        plt.plot(line_x, line_y, c="red", lw=3)
        plt.xlim(xlim); plt.ylim(ylim)
    plt.grid()
    plt.xlabel("Temperatura (°C)"); plt.ylabel("Consumi (GW)")

Usiamo la funzione per visualizzare il modello sovrapposto ai dati su cui è stato addestrato

In [ ]:
plot_model_on_data(X, y, lrm)

Il grafico mostra come il modello lineare sia efficace nell'approssimare la distribuzione dei dati.

## Iperparametri del modello

Quando un modello di regressione viene addestrato, vengono individuati i valori ottimali dei suoi _parametri_: nel caso della regressione lineare, i parametri sono i _coefficienti angolari_ (uno solo nel caso univariato) e l'_intercetta_.

Il processo di addestramento stesso può però avere a sua volta dei parametri che possono essere impostati, detti **_iperparametri_**. Ogni classe di modello di scikit-learn ha diversi iperparametri che possono essere impostati nel momento in cui si istanzia l'oggetto.

Ad esempio `LinearRegression` ha per iperparametro `fit_intercept`, un flag che indica se calcolare un valore per l'intercetta:

- se `True` (default), nell'addestramento si individua un valore ottimale per l'intercetta;
- se `False`, il valore dell'intercetta viene fissato a 0 e vengono ottimizzati solo i coefficienti angolari.

Addestriamo ad esempio un modello con `fit_intercept=False`, specificando tale opzione alla creazione dell'oggetto `LinearRegression`.

In [ ]:
# ni = null intercept
lrm_ni = LinearRegression(fit_intercept=False)

Come sopra, utilizziamo il metodo `fit` per addestrare il modello.

In [ ]:
lrm_ni.fit(X, y)

Stampando i parametri del modello vediamo che l'intercetta è 0.

In [ ]:
lrm_ni.coef_

In [ ]:
lrm_ni.intercept_

Calcoliamo le misure di accuratezza viste sopra (MSE, errore relativo, R²) usando la funzione `print_eval` che abbiamo definito.

In [ ]:
print_eval(X, y, lrm_ni)

- Il modello è leggermente meno accurato di quello con iperparametri default (`fit_intercept=True`), perché è stato rimosso un grado di libertà sull'addestramento
- Creiamo un grafico personalizzato per confrontare i due modelli, si nota che il modello senza intercetta passa per (0, 0)
  - usiamo la funzione `legend` di matplotlib per assegnare un'etichetta a ciascun elemento inserito nel grafico in ordine e visualizzare una legenda

In [ ]:
plt.figure(figsize=(12, 8))
plt.scatter(X, y)
plt.scatter(0, 0, s=100, c="red")
line_x = np.linspace(-2, 32, 100)
line_x_df = pd.DataFrame(line_x[:, None], columns=X.columns)
line_y = lrm.predict(line_x_df)
plt.plot(line_x, line_y, c="green", lw=2)
line_y0 = lrm_ni.predict(line_x_df)
plt.plot(line_x, line_y0, c="red", lw=2)
plt.legend(["Dati", "Origine", "Modello con intercetta", "Modello senza intercetta"])
plt.xlim((-2, 32))
plt.ylim((-0.2, 2.8))
plt.grid()
plt.xlabel("Temperatura (°C)")
plt.ylabel("Consumi (GW)");

## Divisione tra training e test set

Abbiamo visto come addestrare un modello di predizione su un set di dati e validarlo calcolandone diverse metriche **basandosi sugli stessi dati**.

Per verificare se un modello generalizza correttamente i dati su cui è addestrato, è importante **validarlo su dati diversi**.

Il metodo _hold-out_ prevede di dividere i dati a disposizione in

- un _training set_ utilizzato solo per addestrare il modello,
- un _test set_ (o _validation set_) utilizzato solo per calcolare le metriche di accuratezza del modello addestrato.

La proporzione tra training e validation set è arbitraria, possono essere ad es. 50-50 e 66-33.

Avendo dati in ordine cronologico, è comune utilizzare quelli meno recenti come training set e quelli più recenti come test set.

In questo caso scegliamo di utilizzare i dati del 2015 come training set e quelli del 2016 e 2017 come test set.

Definiamo un array booleano che indichi quali dati fanno parte del training set...

In [ ]:
is_train = power_summer.index.year < 2016

Lo usiamo quindi per selezionare i dati del training e del test set (l'operatore `~` esegue il NOT elemento per elemento).

In [ ]:
summer_train = power_summer.loc[is_train]
summer_test = power_summer.loc[~is_train]

## Esercizio 1: Addestramento e validazione modello

- **(1a)** Estrarre esplicitamente il DataFrame della variabile indipendente (`temp`) e la serie della variabile dipendente (`demand`) dai due DataFrame `summer_train` e `summer_val`
- **(1b)** Creare ed addestrare un modello di regressione lineare sul training set `summer_train`
- **(1c)** Stampare le metriche di valutazione viste (MSE, errore relativo, R²) calcolate sullo stesso training set
- **(1d)** Stampare le stesse metriche calcolate sul test set `summer_test`
- **(1e)** Mostrare il grafico del modello sovrapposto al training set
- **(1f)** Mostrare il grafico del modello sovrapposto al test set

In [ ]:
#1a
summer_X_train = summer_train[["temp"]]
summer_y_train = summer_train["demand"]
summer_X_test = summer_test[["temp"]]
summer_y_test = summer_test["demand"]

In [ ]:
#1b
summer_lrm = LinearRegression()
summer_lrm.fit(summer_X_train, summer_y_train)

In [ ]:
#1c
print("Training set:")
print_eval(summer_X_train, summer_y_train, summer_lrm)

In [ ]:
#1d
print("Training set:")
print_eval(summer_X_test, summer_y_test, summer_lrm)

In [ ]:
#1e Mostrare il grafico del modello sovrapposto al training set
plot_model_on_data(summer_X_train, summer_y_train, summer_lrm)

In [ ]:
#1f
plot_model_on_data(summer_X_test, summer_y_test, summer_lrm)

## Caso di studio 2: predizione dei prezzi delle case

Riprendiamo dalla scorsa esercitazione il dataset relativo ai prezzi delle case. Sono fornite informazioni (tasso criminalità, inquinamento, demografia, ...) su diversi quartieri: in base a queste va stimato per ciascuno il prezzo mediano delle abitazioni (`MEDV`).

Contrariamente all'altra volta, forniamo tale dataset già adattato per essere caricato con `read_csv` senza impostare opzioni aggiuntive.

In [ ]:
HOUSING_DATA_URL = "https://github.com/datascienceunibo/dialab2024/raw/main/Regressione_non_Lineare/housing.csv"

import os.path
if not os.path.exists("housing.csv"):
    from urllib.request import urlretrieve
    urlretrieve(HOUSING_DATA_URL, "housing.csv")

In [ ]:
housing = pd.read_csv("housing.csv")

In [ ]:
housing.head(10)

## Divisione training-test con scikit-learn

Abbiamo visto il metodo _hold-out_ in cui i dati disponibili sono divisi in training e test set.

scikit-learn fornisce un'apposita funzione `train_test_split` per suddividere casualmente un set di dati in due insiemi "train" e "test" disgiunti.

- il set di dati è una sequenza di elementi, es. array NumPy o serie/frame pandas
- con l'opzione `test_size=X` si indica la proporzione (se X decimale tra 0 e 1) o il numero di elementi (se X intero) da inserire nel test set (default 0.25, ovvero 25\%)
- tutti i dati non selezionati per il test set sono implicitamente selezionati per il training set
- con l'opzione `random_state` si indica un seed per la riproducibilità: richiamando la funzione più volte con `random_state` e altri parametri uguali viene riprodotta esattamente la stessa suddivisione

Ad esempio, dividiamo il frame `housing` in due parti casuali `train_data` e `test_data`, contenenti rispettivamente 2/3 e 1/3 dei dati.

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(housing, test_size=1/3, random_state=42)

I due nuovi frame contengono due sottoinsiemi disgiunti delle righe di `housing`, mescolate casualmente

In [ ]:
train_data.shape

In [ ]:
train_data.head(3)

In [ ]:
test_data.shape

In [ ]:
test_data.head(3)

A `train_test_split` possiamo passare **due (o più) collezioni** con pari numero di elementi per far sì che siano suddivise insieme, applicando **a tutte lo stesso ordine** casuale dei dati

Ad esempio separiamo `housing` in due strutture dati allineate:

- una serie `y` con i valori della variabile dipendente (`MEDV`),
- un frame `X` con i valori delle variabili indipendenti (tutte le altre).

In [ ]:
y = housing["MEDV"]
X = housing.drop(columns=["MEDV"])

Applichiamo quindi `train_test_split` congiuntamente a `X` e `y`.

In [ ]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=1/3, random_state=42)

Si può verificare dagli indici che l'ordine delle righe di `X_train` è lo stesso di `y_train` (lo stesso vale per `X_test` e `y_test`).

In [ ]:
X_train.head(3)

In [ ]:
y_train.head(3)

In [ ]:
np.array_equal(X_train.index, y_train.index)

## Esercizio 2: Regressione su prezzi case

- **(2a)** Creare ed addestrare un modello di regressione lineare sul training set dei prezzi delle case creato sopra composto da `X_train` e `y_train`
- **(2b)** Stampare le metriche di valutazione viste (MSE, errore relativo, R²) calcolate sullo stesso set
- **(2c)** Stampare le stesse metriche calcolate sul validation set composto da `X_test` e `y_test`

In [ ]:
#2a
lrm = LinearRegression()
lrm.fit(X_train, y_train)

In [ ]:
#2b
print_eval(X_train, y_train, lrm)

In [ ]:
#2c
print_eval(X_test, y_test, lrm)

## Predizione consumi elettrici sull'intero anno

Riprendiamo il caso di studio sulla predizione dei consumi elettrici dalla temperatura.

I modelli addestrati sopra sono addestrati e validati solamente sui mesi estivi.

È possibile addestrare un modello utilizzabile per la predizione sull'intero anno?

Iniziamo dividendo l'intero frame `data` in training e test set ed estraiamo per ciascuno la matrice `X` e il vettore `y`.

In [ ]:
is_train = power.index.year < 2016
X_train = power.loc[is_train, ["temp"]]
y_train = power.loc[is_train, "demand"]
X_test = power.loc[~is_train, ["temp"]]
y_test = power.loc[~is_train, "demand"]

Addestriamo un nuovo modello sul training set estratto.

In [ ]:
lrm = LinearRegression()
lrm.fit(X_train, y_train)

Validiamo il modello sia sullo stesso training set che sul validation set.

In [ ]:
print_eval(X_train, y_train, lrm)

In [ ]:
print_eval(X_test, y_test, lrm)

In entrambi i casi, il modello è poco accurato: lo si deduce in particolare dal coefficiente R² vicino allo zero.

Visualizziamo il modello sovrapposto ai dati (training e test insieme) per indagare...

In [ ]:
plot_model_on_data(power[["temp"]], power["demand"], lrm)

Il grafico evidenzia che l'aumento dei consumi si ha sia nelle giornate più calde (a destra) che in quelle più fredde (a sinistra).

Per questo il consumo non può essere approssimato dalla temperatura in modo soddisfacente con un unico modello lineare (una retta) per tutti i mesi dell'anno.

## Regressione polinomiale

La regressione _polinomiale_ è una generalizzazione di quella lineare in cui il modello include **termini di grado superiore**.

Ad esempio, mentre un modello di regressione lineare su una singola variabile $x$ (_univariata_) ha forma

$$ \hat{y}=\theta_0+\theta_1\cdot x $$

in un modello polinomiale di secondo grado viene aggiunto un termine col quadrato della variabile, con un proprio coefficiente:

$$ \hat{y}=\theta_0+\theta_1\cdot x+\theta_2\cdot x^2 $$

Analogamente, in un modello di terzo grado viene aggiunto un ulteriore termine col cubo della variabile:

$$ \hat{y}=\theta_0+\theta_1\cdot x+\theta_2\cdot x^2+\theta_3\cdot x^3 $$

La regressione polinomiale corrisponde in pratica a quella lineare con l'aggiunta di variabili derivate da quelle esistenti.

Addestriamo ad esempio un modello polinomiale di secondo grado.

Creiamo una matrice `X_train_sq` con i quadrati elemento per elemento di `X_train`.

In [ ]:
X_train_sq = X_train ** 2

Usiamo quindi `np.c_` per unire le due matrici $m\times 1$ in una $m\times 2$.

In [ ]:
X_train_d2 = np.c_[X_train, X_train_sq]

Abbiamo quindi un dataset con due variabili, dove la seconda è il quadrato della prima.

In [ ]:
X_train_d2[:4]

Creiamo come al solito un modello di regressione lineare e addestriamolo con questi dati.

In [ ]:
prm = LinearRegression()
prm.fit(X_train_d2, y_train)

Possiamo già valutare il modello sul training set stesso.

In [ ]:
print_eval(X_train_d2, y_train, prm)

Per valutarlo sul test set, ricaviamo da esso (`X_test`) una matrice `X_test_d2` che includa il quadrato di `X` come fatto prima per il training set.

In [ ]:
X_test_d2 = np.c_[X_test, X_test ** 2]

In [ ]:
print_eval(X_test_d2, y_test, prm)

Vediamo che questo modello con l'aggiunta del termine di secondo grado è molto più accurato di quello lineare.

## Esercizio 3: Regressione polinomiale di terzo grado

- **(3a)** Creare una matrice `X_train_d3` con tre colonne contenenti rispettivamente i valori di `X_train`, i quadrati e i cubi
- **(3b)** Creare una matrice analoga `X_test_d3` basata su `X_test`
- **(3c)** Addestrare un modello di regressione lineare su `X_train_d3`
- **(3d)** Stampare le metriche di valutazione del modello su `X_train_d3` e `X_test_d3`

In [ ]:
#3a
X_train_d3 = np.c_[X_train, X_train ** 2, X_train ** 3]

In [ ]:
#3b
X_test_d3 = np.c_[X_test, X_test ** 2, X_test ** 3]

In [ ]:
#3c
prm = LinearRegression()
prm.fit(X_train_d3, y_train)

In [ ]:
#3d
print_eval(X_train_d3, y_train, prm)
print_eval(X_test_d3, y_test, prm)

## Feature polinomiali con scikit-learn

Per applicare trasformazioni ai dati da fornire ai modelli, scikit-learn fornisce dei _filtri_ con un'interfaccia standard.

Ad esempio, per generare le variabili per la regressione polinomiale, possiamo usare l'apposito filtro `PolynomialFeatures`.

In modo simile ai modelli di predizione, un filtro va dapprima creato, impostandone eventuali iperparametri.

L'iperparametro principale di `PolynomialFeatures` è `degree`, che indica il grado massimo delle feature da generare.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=3)

Come i modelli (es. `LinearRegression`), i filtri vanno "addestrati" con dei dati d'esempio prima di essere utilizzati. Nel caso di `PolynomialFeatures` l'addestramento serve solo a indicare quali variabili sono presenti nei dati in input.

Ad esempio, creiamo una matrice con due osservazioni (righe) di una generica variabile (colonna) $X$.

In [ ]:
sample = np.array([ [ 2],
                    [-3] ])

Passiamo ora questa matrice al metodo `fit`: il filtro apprende che i dati sono composti da una singola variabile.

In [ ]:
poly.fit(sample)

Una volta addestrato, possiamo usare il metodo `transform` del filtro per trasformare qualsiasi matrice compatibile, ovvero con una sola colonna.

Trasformiamo ad esempio la stessa matrice usata in `fit`.

In [ ]:
poly.transform(sample)
#        X^0   X^1   X^2   X^3

Dati i valori di $X$, il filtro ha restituito i valori di $X^0, X^1, X^2, X^3$.

I filtri forniscono un metodo `fit_transform` che raggruppa le due operazioni sopra: addestra il modello sui dati passati (`fit`) e li trasforma (`transform`).

In pratica `fit_transform` va invocato al primo utilizzo di un filtro, mentre per i successivi utilizzi va invocato `transform`.

In [ ]:
poly = PolynomialFeatures(degree=3)
poly.fit_transform(sample)

La colonna $X^0$ ha sempre valore 1 ed è inutile nell'addestrare il modello, in quanto sarebbe ridondante con l'intercetta.

Possiamo escluderla impostando l'iperparametro `include_bias=False` sul filtro.

In [ ]:
poly = PolynomialFeatures(degree=3, include_bias=False)
poly.fit_transform(sample)

Per addestrare un modello polinomiale possiamo quindi creare il filtro per l'aggiunta delle feature, ad esempio di secondo grado...

In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=False)

...e addestrare un modello lineare con i dati trasformati.

In [ ]:
prm = LinearRegression()
X_train_sq = poly.fit_transform(X_train)
prm.fit(X_train_sq, y_train)

In seguito, i valori della _x_ dovranno sempre essere trasformati con lo stesso filtro, utilizzando il metodo `transform`.

Ad esempio, per ottenere i consumi predetti a fronte di temperature di -5 °C, 5 °C e 25 °C...

In [ ]:
sample = [ [-5], [5], [25] ]
sample_sq = poly.transform(sample)
prm.predict(sample_sq)

Per ottenere invece le metriche d'accuratezza...

In [ ]:
X_test_sq = poly.transform(X_test)
print_eval(X_test_sq, y_test, prm)

## Pipeline

Ad ogni interazione col modello `prm` sopra dobbiamo esplicitamente trasformare i dati in ingresso col filtro `poly`: questo rende l'utilizzo del modello più scomodo ed incline ad errori.

Per questi casi, scikit-learn permette di integrare uno o più filtri ed un modello di predizione in un unico blocco, detto **_pipeline_**.

Una volta costruita, la pipeline offre la **stessa API del modello** di predizione incapsulato, ma **applica automaticamente i filtri** ai dati prima di passarli al modello:

- chiamando `fit`, i filtri sono addestrati e applicati (`fit_transform`) sui dati di addestramento;
- chiamando `predict` (o `score`), i filtri già tarati sono applicati ai nuovi dati (`transform`).

Ad esempio, per creare un modello polinomiale facile da usare, incapsuliamo in una pipeline un filtro `PolynomialFeatures` e un modello `LinearRegression`.

Usiamo la classe `Pipeline`, passando al costruttore la lista dei componenti **in ordine di applicazione**. Ciascun componente è rappresentato da una tupla con un nome e la definizione del componente.

In [ ]:
from sklearn.pipeline import Pipeline
prm = Pipeline([
    # nome     elemento
    ("poly",   PolynomialFeatures(degree=2, include_bias=False)),
    ("linreg", LinearRegression())
])

Possiamo ora eseguire le stesse operazioni vista sopra sul modello, ma con la trasformazione dei dati applicata in automatico. Nella rappresentazione grafica del modello è possibile vedere i due componenti.

In [ ]:
# addestramento
prm.fit(X_train, y_train)

In [ ]:
# inferenza
prm.predict([ [-5], [5], [25] ])

In [ ]:
# calcolo accuratezza
print_eval(X_test, y_test, prm)

Tramite l'attributo `named_steps` della pipeline possiamo accedere ai singoli componenti: funziona come un dizionario le cui chiavi sono i nomi assegnati ai componenti nel costruttore.

In [ ]:
prm.named_steps.keys()

Possiamo ad esempio reperire il modello di regressione lineare `linreg`...

In [ ]:
prm.named_steps["linreg"]

...e vedere i pesi assegnati alle due variabili, nell'ordine X e X².

In [ ]:
prm.named_steps["linreg"].coef_

## Curva del modello polinomiale

Visualizzando il modello polinomiale di secondo grado in un grafico, vediamo che ha la forma di una funzione $a\cdot x^2+b\cdot x+c$, ovvero una parabola.

In [ ]:
plot_model_on_data(X_test, y_test, prm)

Come anche confermato dalle misure di errore, questo modello approssima molto meglio il consumo in base alla temperatura rispetto a quello lineare.

## Esercizio 4: Regressione polinomiale con pipeline

- **(4a)** Creare un modello di regressione polinomiale di terzo grado usando una pipeline come sopra
- **(4b)** Addestrare il modello sui dati di training
- **(4c)** Stampare le misure di accuratezza del modello calcolate sui dati di training e su quelli di test
- **(4d)** Visualizzare il grafico del modello sovrapposto ai dati di test

In [ ]:
#4a
prm = Pipeline([
    # nome     elemento
    ("poly",   PolynomialFeatures(degree=3, include_bias=False)),
    ("linreg", LinearRegression())
])

In [ ]:
#4b
prm.fit(X_train, y_train)

In [ ]:
#4c
print_eval(X_train, y_train, prm)
print_eval(X_test, y_test, prm)

In [ ]:
#4d
plot_model_on_data(X_test, y_test, prm)

## Standardizzazione dei dati

Data una variabile $X$ con media $\mu$ e deviazione standard $\sigma$, la sua versione "standard" è calcolata come

$$ Z = \frac{X-\mu}{\sigma} $$

$Z$ mantiene la distribuzione dei dati di $X$, ma con media 0 e deviazione standard 1.

In presenza di più variabili con scale diverse, applicare la standardizzazione a ciascuna di esse è utile per uniformarne i valori: questo può migliorare l'accuratezza dei modelli di regressione.

La standardizzazione si esegue col filtro `StandardScaler`

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

Eseguendo `fit_transform` (o `fit`) il filtro memorizza medie e deviazioni standard dei dati passati, salvate negli attributi `mean_` e `scale_`.

In [ ]:
scaler.fit_transform([[0.001, 4000],
                      [0.002, 2500],
                      [0.004, 1000]])

In [ ]:
scaler.mean_

In [ ]:
scaler.scale_

Successivamente `transform` trasformerà ulteriori dati secondo medie e deviazioni memorizzate.

In [ ]:
scaler.transform([ [0.01, 100] ])

Addestriamo un modello polinomiale di secondo grado come quello sopra, ma applicando la standardizzazione alle variabili $X$ e $X^2$.

Per far ciò, basta aggiungere il filtro alla pipeline.

In [ ]:
prm = Pipeline([
    ("poly",   PolynomialFeatures(degree=2, include_bias=False)),
    ("scale",  StandardScaler()),   # <- aggiunto
    ("linreg", LinearRegression())
])
prm.fit(X_train, y_train)

Valutiamo l'accuratezza del modello.

In [ ]:
print_eval(X_test, y_test, prm)

In questo caso le metriche di accuratezza non sono cambiate standardizzando i dati.

Ma cosa succede con più variabili con ordini di grandezza molto diversi?

## Normalizzazione min-max dei dati

Una tecnica alternativa alla standardizzazione per preprocessare dati in scale diverse è la _normalizzazione minimo-massimo_, che porta i valori di ogni variabile in un intervallo prestabilito, di solito tra 0 e 1.

Data una generica variabile $X$, la sua normalizzazione $N$ si calcola come

$$ N=\frac{X-\min(X)}{\max(X)-\min(X)}$$

Tale trasformazione si effettua col filtro `MinMaxScaler`.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

Addestrando il filtro, vengono salvati gli attributi `min_` ($\min(X)$) e `scale_` ($\frac{1}{\max(X)-\min(X)}$) usati per effettuare la normalizzazione.

In [ ]:
scaler.fit_transform([[0.001, 4000],
                      [0.002, 2500],
                      [0.004, 1000]])

In [ ]:
scaler.min_

In [ ]:
scaler.scale_

## Esercizio 5: Normalizzazione

- **(5a)** Addestrare sul training set un modello di regressione polinomiale di grado 15 (con variabili $X,X^2,\ldots,X^{15}$) e verificarne le misure di accuratezza sul test set
- **(5b)** Ripetere il punto 4a, ma applicando la standardizzazione dei dati nel modello
- **(5c)** Ripetere il punto 4a, ma applicando la normalizzazione min-max dei dati nel modello

Avendo a che fare con molte variabili di ordini di grandezza diversi, la standardizzazione contribuisce a migliorare l'accuratezza

In [ ]:
#5a
prm = Pipeline([
    ("poly",   PolynomialFeatures(degree=15, include_bias=False)),
    ("linreg", LinearRegression())
])
prm.fit(X_train, y_train)
print_eval(X_test, y_test, prm)
plot_model_on_data(X_test, y_test, prm)

In [ ]:
#5b
prm = Pipeline([
    ("poly",   PolynomialFeatures(degree=15, include_bias=False)),
    ("scale",  StandardScaler()), 
    ("linreg", LinearRegression())
])
prm.fit(X_train, y_train)
print_eval(X_test, y_test, prm)
plot_model_on_data(X_test, y_test, prm)

In [ ]:
#5c
prm = Pipeline([
    ("poly",   PolynomialFeatures(degree=15, include_bias=False)),
    ("scale",  MinMaxScaler()), 
    ("linreg", LinearRegression())
])
prm.fit(X_train, y_train)
print_eval(X_test, y_test, prm)
plot_model_on_data(X_test, y_test, prm)